In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bill_pre import *

## 读取账单信息并进行预处理

- 列补充
    - 单独做出年、月、日、时、相对首日的日数这几个列
    - 所有条目按照时间排序
    - 将收入支出合并到金额条目中。为了方便分析，支出为正，收入为负
    - 计算当时总余额
        - （先设置初始为0，然后看总账单余额最低点，再将余额最低点设为0就可以得到近似的余额）
    
- 需要人工
    - 去除付款后退款的条目（相当于交易未发生）：主要是淘宝和携程
    - 合并微信转出后到支付宝花掉的条目
    - 去掉氪金的项目别被发现了
    - 标出自动续费
    - 对商家进行去重
    - 三级标注：项目 - 类别 - 商家

In [13]:
bill = pd.read_excel(r'F:\today\对账\bill_ori.xlsx')
bill.columns = [
    "cmd", # commodity商品
    "cp", # conterparty 交易对方
    "in/ex", # 收入/支出
    "amount", # 金额
    "type", # 交易类型
    "pay_mode", # 支付方式
    "state", # 交易状态
    "date-time", # 日期时间
]
bill.head()

,cmd,cp,in/ex,amount,type,pay_mode,state,date-time
0,转账备注:微信转账,YL尚文轩，jy方肝胆合作师兄,收入,111.0,转账,/,已存入零钱,2024-04-01 22:44:30
1,510441温州永泰店购物,十足便利店,支出,49.0,商户消费,零钱通,支付成功,2024-04-01 22:07:44
2,美团/大众点评点餐订单-04014629709136792857749,巷弄食集,支出,28.0,商户消费,零钱通,支付成功,2024-04-01 12:03:30
3,购买大会员,上海宽娱,支出,15.0,商户消费,零钱通,支付成功,2024-04-01 11:43:10
4,收款方备注:二维码收款,Jackqiu,支出,33.0,扫二维码付款,零钱通,已转账,2024-03-31 23:14:09


In [31]:
# 单独做出年、月、日、时、相对首日的日数这几个列
def get_ymdh(time_stamp, time_ref):
    """
    输入单个时间戳，返回、年、月、日、时、相对首日日数
    ---
    输入
    - time_stamp: 单个时间戳
    - time_ref: 参考时间, 也是一个时间戳
    """
    year = time_stamp.year
    month = time_stamp.month
    day = time_stamp.day
    hour = time_stamp.hour

    time_delta = time_stamp - time_ref
    day_delta = time_delta.days
    return year, month, day, hour, day_delta

time_ref = pd.Timestamp('2024-01-01 00:00:00')
bill['year'], bill['month'], bill['day'], bill['hour'], bill['day_delta'] = zip(*bill['date-time'].apply(lambda x: get_ymdh(x, time_ref)))

In [34]:
# 按时间排序
bill = bill.sort_values(by='date-time')

In [39]:
# 将收入支出合并到金额条目中。为了方便分析，支出为正，收入为负
# （依赖于已经事先删除“转出/转入到支付宝”、“购买后又退款”等特殊类型的数据，否则收支会不止收入/支出两种情况）

# 构造"ine"这个临时的列。如果是收入，则ine=-1；否则ine=1
bill['ine'] = bill['in/ex'].apply(lambda x: -1 if x == '收入' else 1)
bill['amount'] = bill['amount'] * bill['ine']  # 相乘，使得金额获得代表收入/支出的正负符号
bill.drop(columns=['ine', 'in/ex'], inplace=True)  # 删去临时的列

In [41]:
# 计算当时总余额
bill['total_balance'] = bill['amount'].cumsum()

In [33]:
# 去除付款后退款的条目（相当于交易未发生）：主要是淘宝和携程
# 合并微信转出后到支付宝花掉的条目
# 标出自动续费
# 对商家进行去重
# 三级标注：项目 - 类别 - 商家
# 去掉氪金的项目别被发现了


# 输出结果

- 针对总体和任意时间和任意项目
    - 对于每一子类别的饼图
    - 其中金额最大的几笔账目或几个类别的列举
    - 平均每个类别的账目发生的次数和平均金额，中位数、分位数和分布图
    - 总支出的随时间的变化折线图
    - 按月的堆积折线图